# Calculating distances in bulk

In [1]:
import pandas as pd
import numpy as np
from geopy import distance
from geopy.distance import lonlat
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="Test")

itinaries = [("Prittwitzstrasse, Ulm", "Albert-Einstein-allee, Ulm"), ("221B Baker street, London", "Trafalgar Square, London"), ("ENIB, Plouzané", "Place de la Liberté, Brest")]

## Getting the coordinates of the points

First, we need the coordinates of the start and end points of each itinary:

In [2]:
itin_coords = []
for itinary in itinaries:
    start = geolocator.geocode(itinary[0])
    start = (start.latitude, start.longitude)
    end = geolocator.geocode(itinary[1])
    end = (end.latitude, end.longitude)
    itin_coords.append((start, end))

itin_coords

[((48.4114815, 9.9922708), (48.4268787, 9.9566362)),
 ((51.5237604, -0.1584945), (51.508037, -0.128049410707247)),
 ((48.36153155, -4.5662882688034), (48.3908095, -4.48570395555493))]

## Getting the distance as the crow flies

In [3]:
distances_crow = [distance.geodesic(i[0], i[1], ellipsoid='GRS-80').km for i in itin_coords]
distances_crow

[3.144604427791515, 2.7434743924747442, 6.799817119749467]

## Getting the distance by route

In [4]:
from pyroutelib2.loadOsm import LoadOsm
from pyroutelib2.route import Router

data = LoadOsm("cycle")
router = Router(data)

def distance_by_route(start, end):
    # Change coordinates into nodes :
    start_node = data.findNode(start[0], start[1])
    end_node = data.findNode(end[0], end[1])

    # Calculating the route :
    result, route = router.doRoute(start_node, end_node)
    lats = []
    lons = []
    if result == 'success':
        for i in route:
            node = data.rnodes[i]
            lats.append(node[0])
            lons.append(node[1])
    else:
        print("Error calculating the route.", start_node)
    
    # Getting the distance between each points :
    distance_route = 0
    for i in range(len(lons)-1):
        p1=(lats[i],lons[i])
        p2=(lats[i+1],lons[i+1])
        distance_route += distance.geodesic(p1,p2, ellipsoid='GRS-80').km
    
    return distance_route

In [5]:
distances = [distance_by_route(i[0], i[1]) for i in itin_coords]
distances

ParseError: unclosed token: line 78725, column 2 (<string>)